# Расчет данных по нескольким ЦА

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Создаем объекты для работы с API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()

# Общие параметры для заданий

Для похожих отчетов часто используются одни и те же параметры, удобно их вынести в отдельные переменные, и затем использовать в коде. Если вдруг понадобится какой-то параметр изменить, его нужно поменять только в одном месте.

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2021-09-01', '2021-09-30')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по медиа, в нашем случае это ID ресурса Avito
mart_filter = 'crossMediaResourceId = 1028'

# Задаем необходимые группы
groups = {'12+': 'ageGroup IN (1, 2, 3, 4, 5, 6, 7)',
          '12-24':'ageGroup IN (1, 2)', 
          '25-34':'ageGroup = 3', 
          '35-44':'ageGroup = 4', 
          '45-54':'ageGroup = 5', 
          '55+':'ageGroup IN (6, 7)', 
          'Муж.12+':'sex = 1', 
          'Жен.12+':'sex = 2'}

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaResourceId", "useTypeName"]

# Указываем список статистик для расчета
statistics = ['reach']

# Расчет для нескольких ЦА

In [ ]:
%%time

# Посчитаем данные в циклах
tasks = []
print('Отправка задач ')

for agegroup, logical in groups.items():
    
    # Формируем задание на расчет ЦА
    project_name = f"{agegroup}" 
    demo_filter = f"{logical}"

    # Формируем задание для API Cross Web в формате JSON
    task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                                 demo_filter, mart_filter, slices, statistics)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name
    tsk['task'] = mtask.send_audience_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '') 

print('')
        
# Ждем выполнения
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')        
        
# Получаем результат
results = []
for t in tasks:
    tsk = t['task']
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])
    results.append(df_result)
df_result = pd.concat(results)
df_result       

In [ ]:
# Сохраняем в Excel
writer = pd.ExcelWriter(mtask.get_excel_filename('several-target-audiences'))
df_result.to_excel(writer, 'Report', index=False)
writer.save()